In [1]:
from strands import Agent
from strands_tools import agent_graph
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient
from strands_tools import workflow
from mcp.client.streamable_http import streamablehttp_client

In [5]:
import os
import base64
##https://github.com/awslabs/amazon-bedrock-agent-samples/tree/main/examples/agent_observability/deploy-langfuse-on-ecs-fargate-with-typescript-cdk

secret_key="sk-lf-46839148-58c4-4cef-9115-75623c78553a"
public_key="pk-lf-0a30d748-a9b9-4325-9fd6-fa09a44ff3b5"
os.environ["LANGFUSE_HOST"] = "https://d3og9zzfs5n481.cloudfront.net" 

# Set up endpoint
otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"

# Create authentication token:

auth_token = base64.b64encode(f"{public_key}:{secret_key}".encode()).decode()
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"

In [ ]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="npx", args=["-y", "tavily-mcp@0.1.3"],env={"TAVILY_API_KEY": "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"})
))

streamable_http_mcp_client = MCPClient(lambda: streamablehttp_client(
    url="https://har89we10a.execute-api.us-east-1.amazonaws.com/Prod/mcp",
    headers={"Authorization":"Bearer 123456"}))

# Create an agent with MCP tools
with streamable_http_mcp_client:
    with stdio_mcp_client:
        tools = stdio_mcp_client.list_tools_sync()+streamable_http_mcp_client.list_tools_sync()
        for tool in tools:
            print(tool.tool_name, tool.tool_spec)
        # Initialize and create a multi-agent system
        agent = Agent(tools=[agent_graph]+tools,
                          trace_attributes={
                        "session.id": "abc-1234",
                        "user.id": "river@domain.com",
                        "tags": [
                            "Agent-SDK",
                            "Okatank-Project",
                            "Observability-Tags",
                        ]
    },
                      )
        result = agent.tool.agent_graph(
            action="create",
            graph_id="research_team",
            topology={
                "type": "star",
                "nodes": [
                    {
                        "id": "coordinator",
                        "role": "team_lead",
                        "system_prompt": "You are a research team leader coordinating specialists."
                    },
                    {
                        "id": "data_analyst",
                        "role": "analyst",
                        "system_prompt": "You are a data analyst specializing in statistical analysis. you can use get time tool to get current time and tavily tools to get latest data"
                    },
                    {
                        "id": "domain_expert",
                        "role": "expert",
                        "system_prompt": "You are a domain expert with deep subject knowledge."
                    }
                ],
                "edges": [
                    {"from": "coordinator", "to": "data_analyst"},
                    {"from": "coordinator", "to": "domain_expert"},
                    {"from": "data_analyst", "to": "coordinator"},
                    {"from": "domain_expert", "to": "coordinator"}
                ]
            }
        )

        # Send a task and check status
        agent.tool.agent_graph(
            action="message",
            graph_id="research_team",
            message={
                "target": "coordinator",
                "content": """Analyze AI impacts human relationships"""
            }
        )
        status = agent.tool.agent_graph(action="status", graph_id="research_team")

Error parsing JSON response: 1 validation error for JSONRPCMessage
  Invalid JSON: EOF while parsing a value at line 1 column 0 [type=json_invalid, input_value=b'', input_type=bytes]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid


tavily-search {'inputSchema': {'json': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query'}, 'search_depth': {'type': 'string', 'enum': ['basic', 'advanced'], 'description': "The depth of the search. It can be 'basic' or 'advanced'", 'default': 'basic'}, 'topic': {'type': 'string', 'enum': ['general', 'news'], 'description': 'The category of the search. This will determine which of our agents will be used for the search', 'default': 'general'}, 'days': {'type': 'number', 'description': "The number of days back from the current date to include in the search results. This specifies the time frame of data to be retrieved. Please note that this feature is only available when using the 'news' search topic", 'default': 3}, 'time_range': {'type': 'string', 'description': "The time range back from the current date to include in the search results. This feature is available for both 'general' and 'news' search topics", 'enum': ['day', 'week', 'month', 'ye

Session termination failed: 403


# Analysis of AI's Impact on Human Relationships

## Research Framework

To analyze how artificial intelligence impacts human relationships, we need to examine multiple dimensions of this complex interaction.

### Key Dimensions for Investigation

**1. Communication Patterns**
- How AI-mediated communication (chatbots, virtual assistants) changes interpersonal dynamics
- Effects of algorithm-curated content on relationship formation and maintenance
- Translation and accessibility tools bridging language and ability gaps

**2. Psychological Effects**
- Changes in intimacy formation with AI presence
- Potential displacement of human-to-human interaction
- Evolving emotional expectations and satisfaction metrics

**3. Social Structures**
- Dating and matchmaking algorithmic influence
- Family dynamics and AI integration in domestic spaces
- Workplace relationship transformations through AI collaboration tools

**4. Ethical Considerations**
- Privacy boundaries within relationships influen

In [7]:
agent.tool.agent_graph(action="stop", graph_id="research_team")

{'toolUseId': 'tooluse_agent_graph_515340774',
 'status': 'success',
 'content': [{'text': 'Graph research_team stopped and removed'}]}